In [3]:
import numpy as np
import pandas as pd

In [9]:
data=pd.read_csv('RESSET_MRESSTK_1.csv',usecols=[0,2,3,4,5,6],encoding='GB2312')
data.columns=['code','date','close','tshare','monret','pe']
for i in range(2,7):
    filename='RESSET_MRESSTK_'+str(i)+'.csv'
    trans=pd.read_csv(filename,usecols=[0,2,3,4,5,6],encoding='utf-8')
    trans.columns=['code','date','close','tshare','monret','pe']
    data=pd.concat([data,trans])
data

,code,date,close,tshare,monret,pe
0,1,2000-01-28,18.53,1.071634e+09,0.0619,51.79
1,1,2000-02-29,18.32,1.071634e+09,-0.0113,51.20
2,1,2000-03-31,18.37,1.071634e+09,0.0027,51.34
3,1,2000-04-28,19.05,1.071634e+09,0.0370,56.44
4,1,2000-05-31,18.00,1.071634e+09,-0.0551,53.33
...,...,...,...,...,...,...
92793,990018,2006-05-29,15.47,1.804400e+09,0.4365,52.96
92794,990018,2006-06-30,17.15,1.804400e+09,0.1086,58.71
92795,990018,2006-07-31,16.96,1.804400e+09,-0.0111,34.15
92796,990018,2006-08-31,16.32,1.804400e+09,-0.0377,32.86


In [55]:
data['date'] = pd.to_datetime(data['date'])
data['yearmonth'] = data['date'].dt.strftime('%Y%m').astype(int)
data['tstksize'] = data['close']*data['tshare']
data['stkep'] = 1/data['pe']
#data.dropna(inplace = True, subset=['tstksize', 'stkep'])##直接全部drop不是更好？ 597685行  和下面这代码不一样结果
##data.dropna(inplace = True)
data.dropna(inplace = True)
data

,code,date,close,tshare,monret,pe,yearmonth,tstksize,stkep
0,1,2000-01-28,18.53,1.071634e+09,0.0619,51.79,200001,1.985739e+10,0.019309
1,1,2000-02-29,18.32,1.071634e+09,-0.0113,51.20,200002,1.963234e+10,0.019531
2,1,2000-03-31,18.37,1.071634e+09,0.0027,51.34,200003,1.968592e+10,0.019478
3,1,2000-04-28,19.05,1.071634e+09,0.0370,56.44,200004,2.041464e+10,0.017718
4,1,2000-05-31,18.00,1.071634e+09,-0.0551,53.33,200005,1.928942e+10,0.018751
...,...,...,...,...,...,...,...,...,...
92793,990018,2006-05-29,15.47,1.804400e+09,0.4365,52.96,200605,2.791407e+10,0.018882
92794,990018,2006-06-30,17.15,1.804400e+09,0.1086,58.71,200606,3.094546e+10,0.017033
92795,990018,2006-07-31,16.96,1.804400e+09,-0.0111,34.15,200607,3.060262e+10,0.029283
92796,990018,2006-08-31,16.32,1.804400e+09,-0.0377,32.86,200608,2.944781e+10,0.030432


In [56]:
uym = np.unique(data['yearmonth'].values)##有没有可能有些code不全有date。这没什么关系，因为后面是根据每个时点作为一组，在其中进行排序划分
print(len(uym))
uym

264
[2, 3]


In [63]:
class sort_portfolio:

    def __init__(self, data, months, gnum):
        self.data = data
        self.months = months
        self.gnum = gnum

    def data_months(self):
        dm = self.data.loc[self.data['yearmonth'] == self.months[0], ['code', 'tstksize', 'stkep']]
        dm.dropna(inplace = True)
        for i in range(1, len(self.months)):
            ind = self.data['yearmonth'] == self.months[i]
            dm = pd.merge(left = dm,
                         right = self.data.loc[ind, ['code', 'monret']],
                         on='code',
                         how='left',
                         sort=True)
        dm.columns = ['stk', 'size6', 'ep6', 'ret7', 'ret8', 'ret9', 'ret10', 'ret11',
                      'ret12', 'retn1', 'retn2', 'retn3', 'retn4', 'retn5', 'retn6']
        return dm
    def sort_single_ind(self):
        L = np.sum(self.data['yearmonth'] == self.months[0])##算出有多少个股票
        n = np.fix(L/self.gnum).astype(int)
        x = np.ones(L)
        i = 0
        while i < self.gnum:
            if i == self.gnum-1:
                x[i*n:] = x[i*n:]*i##当i == self.gnum-1，就说明到最后一组了，很有可能后面不够了。但是其实没有关系，因为就算超过最大值了，也只会取到最后的值
            else:
                x[i*n:(i+1)*n] = x[i*n:(i+1)*n]*i
            i = i+1
        ssi = x.astype(int)
        return ssi

    def sort_double_ind(self):
        L = np.sum(self.data['yearmonth'] == self.months[0])
        l = np.fix(L/self.gnum).astype(int)
        n = np.fix(L/(self.gnum**2)).astype(int)
        x = np.ones(L)
        i = 0
        while i < self.gnum:
            j = 0
            while j < self.gnum:
                if j == self.gnum-1:
                    if i == self.gnum-1:
                        x[(i*l+j*n):] = x[(i*l+j*n):]*j
                    else:
                        x[(i*l+j*n):((i+1)*l)] = x[(i*l+j*n):((i+1)*l)]*j
                else:
                    x[(i*l+j*n):(i*l+(j+1)*n)] = x[(i*l+j*n):(i*l+(j+1)*n)]*j
                j=j+1
            i=i+1
        sdi = x.astype(int)
        return sdi

    def sequence_sort(self):
        dm = self.data_months()
        ssi = self.sort_single_ind()
        sdi = self.sort_double_ind()
        dm.sort_values(by=['size6'], ascending=True, inplace=True)
        dm['sinsort'] = ssi
        dm.sort_values(by=['sinsort', 'ep6'], ascending=[True, True], inplace=True)
        dm['dousort'] = sdi
        return dm


    def sequence_sort_mreturn(self):
        sp = self.sequence_sort()
        spmreturn = sp.loc[:, ['ret7', 'sinsort', 'dousort']].dropna().groupby(
                    by=['sinsort', 'dousort'])['ret7'].mean()
        lret = ['ret8', 'ret9', 'ret10', 'ret11', 'ret12', 'retn1', 'retn2', 'retn3', 'retn4', 'retn5', 'retn6']
        for i in lret:
            a = sp.loc[:, [i, 'sinsort', 'dousort']].dropna().groupby(
                by=['sinsort', 'dousort'])[i].mean()
            spmreturn = pd.concat([spmreturn, a], axis=1)
        spmreturn['mret'] = spmreturn.apply(lambda x: x.mean(), axis=1)
        return spmreturn

In [64]:
print(uym[5:5+13])
sp = sort_portfolio(data, uym[5:5+13], 5)
sp.data_months()

[200006 200007 200008 200009 200010 200011 200012 200101 200102 200103
 200104 200105 200106]


E:\Python37\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Passing 'suffixes' which cause duplicate columns {'monret_x'} in the result is deprecated and will raise a MergeError in a future version.
  app.launch_new_instance()


,stk,size6,ep6,ret7,ret8,ret9,ret10,ret11,ret12,retn1,retn2,retn3,retn4,retn5,retn6
0,1,1.942873e+10,0.018619,0.0210,-0.0411,-0.0445,0.0348,-0.0081,-0.0626,0.0317,-0.0594,0.1512,-0.0425,0.0431,-0.0549
1,2,5.386597e+09,0.031437,0.0274,-0.0483,-0.0528,0.0671,0.0606,-0.0007,0.0672,-0.0777,0.0988,-0.0364,-0.0261,0.0430
2,4,9.068779e+08,0.005057,0.2618,0.1707,0.0068,-0.1572,0.1719,0.0319,-0.1515,-0.1071,0.1432,0.0101,-0.0031,0.0083
3,5,2.547178e+09,0.013019,-0.0357,-0.0064,-0.1565,0.0592,0.0522,-0.0371,0.1177,-0.0938,0.1137,NaN,NaN,NaN
4,6,2.122505e+09,0.024839,0.0468,-0.1137,-0.1154,0.0429,0.0691,0.0810,0.0379,-0.0452,0.0695,0.0236,-0.0265,-0.0007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,900950,2.544366e+07,-0.016466,0.1579,0.1273,0.1048,0.0730,0.0544,0.1677,-0.0387,-0.0632,0.8405,0.3567,0.4079,-0.0881
946,900951,1.500000e+07,-0.020859,0.3333,0.4000,-0.1714,0.2155,0.0071,0.3028,-0.0081,0.0054,0.8943,0.2203,0.3916,-0.0110
947,900952,4.173600e+07,0.065833,0.0372,-0.1436,-0.0299,0.0864,0.1023,0.3041,-0.0316,0.0857,0.9718,0.3060,0.3022,-0.0767
948,900953,5.376000e+07,0.055279,0.1518,-0.0233,0.0317,0.1000,0.0350,0.2230,-0.0608,0.0794,0.8801,0.2768,0.2622,-0.0468


In [65]:
sp.sequence_sort()

E:\Python37\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Passing 'suffixes' which cause duplicate columns {'monret_x'} in the result is deprecated and will raise a MergeError in a future version.
  app.launch_new_instance()


,stk,size6,ep6,ret7,ret8,ret9,ret10,ret11,ret12,retn1,retn2,retn3,retn4,retn5,retn6,sinsort,dousort
915,900915,8.142000e+06,-0.990099,0.3051,0.3506,0.0192,0.0943,0.2759,0.2635,-0.0909,-0.0471,0.5833,-0.0351,NaN,NaN,0,0
925,900926,1.038400e+07,-0.719424,0.3898,0.6707,0.0438,-0.0280,0.2014,0.4910,-0.0542,0.0276,1.1736,0.1711,0.1031,0.1001,0,0
907,900906,1.585584e+07,-0.469484,0.2424,0.3049,-0.0280,0.0865,0.2566,0.2535,-0.0787,-0.0366,0.7184,0.3904,0.3921,-0.1047,0,0
434,200025,4.752000e+07,-0.315457,0.0000,0.0944,-0.0761,-0.0055,0.0000,0.2707,-0.0174,-0.1018,1.4877,0.3366,0.6800,-0.0741,0,0
927,900928,1.992906e+07,-0.243902,0.1183,0.2500,0.0615,0.1232,0.0258,0.3396,-0.1127,0.0238,0.6744,0.1713,0.3926,-0.1126,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,825,2.463750e+09,0.047237,0.0898,-0.0363,-0.0565,0.0046,0.0015,-0.0107,0.0498,-0.0134,0.0571,0.0213,-0.0128,0.0086,4,4
190,629,2.122584e+09,0.049554,0.1596,-0.0749,-0.0972,0.0942,0.0219,-0.0294,0.1143,0.0198,0.0703,-0.0838,-0.0235,0.0468,4,4
241,709,2.180531e+09,0.053248,0.0851,-0.0229,-0.0642,0.0106,0.0705,-0.0244,0.0350,0.0217,0.0567,-0.0604,-0.0155,0.0411,4,4
302,800,3.341520e+09,0.054645,0.1879,-0.0934,-0.0923,-0.0051,0.0988,0.0031,0.0912,-0.0212,0.0810,-0.0643,0.0014,-0.0271,4,4


In [66]:
sp.sequence_sort_mreturn()

E:\Python37\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Passing 'suffixes' which cause duplicate columns {'monret_x'} in the result is deprecated and will raise a MergeError in a future version.
  app.launch_new_instance()


ret7      ret8      ret9     ret10     ret11     ret12  \
sinsort dousort                                                               
0       0        0.151432  0.133645  0.009450  0.061500  0.070413  0.082708   
        1        0.067492  0.045516 -0.040279  0.070713  0.063850  0.044595   
        2        0.029908  0.026773 -0.023862  0.077095  0.062868  0.075029   
        3        0.058742  0.033461 -0.028766  0.074634  0.069924  0.127318   
        4        0.044184  0.020995 -0.077255  0.084105  0.064945  0.172776   
1       0        0.139982  0.052213 -0.037438  0.066354  0.051376 -0.022311   
        1        0.052395  0.032505 -0.016979  0.044368  0.078127 -0.007886   
        2        0.047634  0.017376 -0.034524  0.058347  0.049937  0.020853   
        3        0.041529  0.012916 -0.026413  0.048668  0.093179  0.020676   
        4        0.051768 -0.014042 -0.051671  0.058637  0.063911  0.029542   
2       0        0.032176  0.068400 -0.013695  0.051521  0.064137 -0.012266   
        1        0.031382 -0.005650 -0.040224  0.022126  0.077345 -0.017071   
        2        0.042134  0.005171 -0.035189  0.038395  0.057742  0.016495   
        3        0.040089 -0.000326 -0.042395  0.032884  0.063253  0.013545   
        4        0.086755 -0.036142 -0.047718  0.040624  0.067139  0.025971   
3       0        0.049642 -0.009334 -0.058713  0.040889  0.066166 -0.011626   
        1        0.023366 -0.014982 -0.042455  0.008218  0.059326  0.008421   
        2        0.041445 -0.008721 -0.045053  0.048074  0.058126  0.004484   
        3        0.033587 -0.018963 -0.069684  0.036171  0.066550  0.012876   
        4        0.052789 -0.031113 -0.067166  0.018845  0.071355  0.023074   
4       0        0.019876 -0.022721 -0.040434  0.009903  0.035026 -0.036408   
        1        0.023032 -0.045750 -0.045716 -0.002313  0.057205 -0.004137   
        2        0.038424  0.008182 -0.032032  0.001118  0.043308 -0.018629   
        3        0.035005 -0.024808 -0.057897  0.007382  0.040566 -0.003797   
        4        0.078050 -0.019332 -0.082408  0.015800  0.071679  0.001063   

                    retn1     retn2     retn3     retn4     retn5     retn6  \
sinsort dousort                                                               
0       0       -0.044237 -0.067926  0.427571  0.114558  0.205778 -0.021728   
        1       -0.044616 -0.054645  0.240392  0.076316  0.153997 -0.035558   
        2       -0.038147 -0.043892  0.315939  0.064513  0.141132 -0.021161   
        3       -0.032697 -0.012624  0.551766  0.075342  0.196279 -0.041282   
        4       -0.010937  0.045863  1.081932  0.138605  0.244211 -0.098176   
1       0       -0.012237 -0.100247  0.182011  0.027332  0.117717 -0.011750   
        1       -0.026987 -0.065621  0.101492  0.022968  0.079986  0.009749   
        2        0.000324 -0.074274  0.079034  0.015847  0.071192  0.018216   
        3       -0.007818 -0.049808  0.105650  0.010700  0.073211  0.011624   
        4       -0.000474 -0.036134  0.284471  0.013747  0.106308 -0.011876   
2       0       -0.027361 -0.089711  0.100082  0.021726  0.059781 -0.009105   
        1       -0.000982 -0.081176  0.081416 -0.006308  0.070637  0.006239   
        2       -0.016908 -0.059616  0.068332 -0.005037  0.058579  0.028647   
        3        0.006795 -0.062853  0.095271 -0.013061  0.045989  0.018905   
        4        0.004642 -0.034387  0.068987 -0.019026  0.049703  0.028918   
3       0        0.004497 -0.054484  0.080282 -0.010816  0.041984 -0.003674   
        1       -0.016442 -0.053763  0.066024  0.006371  0.038779 -0.014021   
        2        0.005274 -0.063479  0.064268 -0.014379  0.054295  0.006526   
        3        0.010989 -0.051245  0.069721 -0.025503  0.027750  0.007626   
        4        0.013471 -0.038239  0.080908 -0.034516  0.023505  0.030884   
4       0       -0.015047 -0.048200  0.046708 -0.022529 -0.004724  0.014103   
        1       -0.029434 -0.081808  0.071092  0.004408 -0.012157 -0.0

In [68]:
meanret = []
lcname = []
for i in range(5, 258, 12):
    if len(uym[i:i+13]) == 13:
        lcname.append(str(uym[i]))
        sp = sort_portfolio(data, uym[i:i+13], 5)
        spmreturn = sp.sequence_sort_mreturn()
        if len(meanret) == 0:
            meanret = spmreturn['mret']
        else:
            meanret = pd.concat([meanret, spmreturn['mret']], axis=1)
meanret.columns = lcname
meanret

E:\Python37\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Passing 'suffixes' which cause duplicate columns {'monret_x'} in the result is deprecated and will raise a MergeError in a future version.
  app.launch_new_instance()


200006    200106    200206    200306    200406    200506  \
sinsort dousort                                                               
0       0        0.093597 -0.021995 -0.026883 -0.023502 -0.037301  0.046782   
        1        0.048981 -0.023297 -0.023558 -0.019235 -0.031713  0.042067   
        2        0.055516 -0.023141 -0.025105 -0.020707 -0.033495  0.043425   
        3        0.089341 -0.019015 -0.022886 -0.016855 -0.029608  0.038498   
        4        0.142604 -0.022905 -0.019197 -0.011121 -0.022004  0.034566   
1       0        0.037750 -0.021207 -0.025861 -0.022200 -0.045921  0.043654   
        1        0.025343 -0.019438 -0.027229 -0.017842 -0.031102  0.039372   
        2        0.022497 -0.017301 -0.024836 -0.018014 -0.037441  0.039598   
        3        0.027843 -0.019848 -0.022047 -0.012902 -0.026114  0.038767   
        4        0.041182 -0.017743 -0.017486 -0.006732 -0.016443  0.045876   
2       0        0.020474 -0.020476 -0.026063 -0.023374 -0.042902  0.036110   
        1        0.011478 -0.022425 -0.021050 -0.018822 -0.028126  0.042113   
        2        0.016562 -0.023573 -0.020560 -0.018332 -0.027272  0.035675   
        3        0.016508 -0.018726 -0.017442 -0.009355 -0.020101  0.050329   
        4        0.019622 -0.018544 -0.013345 -0.001332 -0.015303  0.040701   
3       0        0.011234 -0.015985 -0.028253 -0.017372 -0.028746  0.033861   
        1        0.005737 -0.022469 -0.019021 -0.014911 -0.031290  0.044497   
        2        0.012572 -0.017929 -0.020070 -0.011735 -0.032414  0.048638   
        3        0.008323 -0.018186 -0.020186 -0.005392 -0.018992  0.040353   
        4        0.011983 -0.014322 -0.006049 -0.000020 -0.018392  0.033529   
4       0       -0.005371 -0.019442 -0.018912 -0.024081 -0.028089  0.037185   
        1       -0.005872 -0.018993 -0.022213 -0.010271 -0.020710  0.042642   
        2       -0.001578 -0.012820 -0.014503 -0.007793 -0.016512  0.036348   
        3       -0.001270 -0.011244 -0.008988 -0.000310 -0.009637  0.034580   
        4        0.008622 -0.009237 -0.004910  0.006356 -0.009874  0.027724   

                   200606    200706    200806    200906  ...    201106  \
sinsort dousort                                          ...             
0       0        0.109080 -0.001380  0.055228  0.015534  ... -0.007321   
        1        0.111025 -0.004689  0.046802  0.022339  ... -0.009218   
        2        0.097414  0.000050  0.050527  0.024425  ... -0.008531   
        3        0.074116  0.003206  0.052264  0.023590  ... -0.010284   
        4        0.094213 -0.002541  0.027980  0.018361  ... -0.010843   
1       0        0.089824 -0.002921  0.047789  0.004571  ... -0.021244   
        1        0.091156 -0.005369  0.045643  0.017066  ... -0.018245   
        2        0.075056 -0.001729  0.046631  0.018329  ... -0.011660   
        3        0.079919 -0.001796  0.049728  0.028495  ... -0.009038   
        4        0.073499  0.003251  0.045131  0.017298  ... -0.013687   
2       0        0.090917 -0.008944  0.045795  0.008929  ... -0.026264   
        1        0.076479 -0.007201  0.039690  0.007526  ... -0.015175   
        2        0.074638 -0.002289  0.037931  0.020604  ... -0.011911   
        3        0.087001 -0.001589  0.042958  0.026219  ... -0.015023   
        4        0.083209 -0.001780  0.045418  0.020147  ... -0.017455   
3       0        0.080814 -0.018624  0.030866  0.004419  ... -0.025258   
        1        0.082026 -0.022995  0.023195  0.011340  ... -0.017042   
        2        0.066871 -0.010255  0.025126  0.008861  ... -0.016899   
        3        0.079886 -0.004730  0.040476  0.018073  ... -0.015669   
        4        0.086323 -0.004943  0.034018  0.010521  ... -0.019293   
4       0        0.082050 -0.026597  0.023287 -0.003168  ... -0.022930   
        1        0.082713 -0.012998  0.019979 -0.008540  ... -0.013615   
        2        0.092741 -0.018992  0.028091 -0.003400  ... -0.016747   
        3        0.092762 -0.007688

In [69]:
meanret['meanreturn'] = meanret.apply(lambda x: x.mean(), axis=1)
a = meanret['meanreturn'].values.reshape((5,5))
a

array([[0.02168166, 0.01944209, 0.02049765, 0.02059922, 0.01964971],
       [0.01182647, 0.01264529, 0.01344856, 0.01472884, 0.01635444],
       [0.00771147, 0.00891601, 0.01123481, 0.01430282, 0.01360664],
       [0.00567476, 0.00669084, 0.00797399, 0.01060283, 0.01184845],
       [0.00431894, 0.00621612, 0.00723364, 0.00927496, 0.01075246]])

In [70]:
print('{:>10s} {:>10s}, {:>10s}, {:>10s}, {:>10s}, {:>10s}'.format('', 'EP1', 'EP2', 'EP3', 'EP4', 'EP5'))
for i in range(5):
    print('{:>10s} {:10.5f}, {:10.5f}, {:10.5f}, {:10.5f}, {:10.5f}'.format('SIZE'+str(i+1),
                                                                            a[i, 0],
                                                                            a[i, 1],
                                                                            a[i, 2],
                                                                            a[i, 3],
                                                                            a[i, 4]))

                  EP1,        EP2,        EP3,        EP4,        EP5
     SIZE1    0.02168,    0.01944,    0.02050,    0.02060,    0.01965
     SIZE2    0.01183,    0.01265,    0.01345,    0.01473,    0.01635
     SIZE3    0.00771,    0.00892,    0.01123,    0.01430,    0.01361
     SIZE4    0.00567,    0.00669,    0.00797,    0.01060,    0.01185
     SIZE5    0.00432,    0.00622,    0.00723,    0.00927,    0.01075


In [12]:
m=pd.DataFrame({'a':[pd.NA,2,3],'b':[5,6,8]})
m

,a,b
0,<NA>,5
1,2,6
2,3,8


In [14]:
l=m.dropna()
l

,a,b
1,2,6
2,3,8
